In [ ]:
import importlib as ilib
import logging
import plotly.offline as plo
import plotly.graph_objs as go
import time

In [ ]:
import omega.configuration as oc
import omega.core.chain as occ
import omega.core.chart as chart
import omega.core.instrument as oci
import omega.raid.backtrade as orb
import omega.raid.events as ore
import omega.raid.tests as ort
import omega_ui.tearsheet as ots

plo.init_notebook_mode(connected=True)

In [ ]:
# Disable logging output
log = logging.getLogger()
log.setLevel(logging.WARNING)

In [ ]:
import omega.raid.utils as oru
import omega_ui.backtest as ob
ilib.reload(occ)
ilib.reload(oru)

# Backtest

In [ ]:
import alpha.forecasts as af
import alpha.harvester as ah
import alpha.roll_return as arr
import alpha.spreading as asp
import alpha.risk as ar
ilib.reload(orb)
ilib.reload(arr)
ilib.reload(ah)
start = time.time()
fbt = orb.FuturesBacktest()
# Markets and Cash
# ['BO', 'C_', 'CC', 'CL', 'CT', 'FC', 'GC', 'HG', 'HO', 'KC', 'LB', 'LC', 'LH', 'NG', 'O_', 'OJ', 'PA', 'RB', 'RR', 'S_', 'SB', 'SI', 'SM', 'W_']  # TODO: Fix PL
# 'BO', 'C_', 'CC', 'CL', 'CT', 'FC', 'HG', 'HO', 'KC', 'LB', 'LC', 'LH', 'O_', 'OJ', 'RR', 'SB', 'SI', 'W_'
# ['FC', 'HO', 'KC', 'LB', 'LC', 'LH', 'O_', 'OJ', 'RR', 'SB']
# Summary: Problem with position across strategies Solutions? - easiest disable exit with front if more than one strategy - harder: keep track of positions in the strategy? Or move to mono strategy (would be faster to implement and run)?
stems = ['ES']
cash = 1000000.0
# Strategy
strategies = [ah.Front]  # ah.Front, ah.Reversal
strategies = [{'strategy': s, 'parameters': fbt.get_parameters(s, stems)} for s in strategies]
# If we need to optimize
optimization = True
if not optimization:
    pnl, results = fbt.run_multi(stems, cash, strategies)
    print('Total Profit: {:,.2f}'.format(pnl))
    print('Backtest - Execution time: {:.0f} seconds!'.format(time.time() - start))
else:
    log.setLevel(logging.ERROR)
    optim_range = range(5, 15)
    # Always optimize 1st strategy!
    strategy = strategies[0]['strategy']
    params = strategies[0]['parameters']
    params['optim_value'] = optim_range
    pnls, results = orb.optimize(stems, cash, strategy, params)
    dfo = pd.DataFrame.from_dict(pnls)
    dfo.index = optim_range
    print('Optimization values:\n{}'.format(dfo[['PnL', 'DD', 'Sharpe']]))
    print('Optimization - Execution time: {:.0f} seconds!'.format(time.time() - start))
    log.setLevel(logging.INFO)

## Tearsheet

In [ ]:
ilib.reload(ots)
# Create Tearsheet
tearsheet = ots.create_tearsheet(results, '{}: {:,.2f}'.format(stems, pnl), notebook=True)
# create graphsn
plo.iplot(tearsheet['fig'])
# print statistics
for section in tearsheet['statistics']:
    print(section)
    for stat in tearsheet['statistics'][section]:
        print('   %s: %s' % (stat, tearsheet['statistics'][section].get(stat)))

In [ ]:
# Gross Leverage 
dfl = tearsheet['gross_leverage']
plo.iplot({
    'data': [go.Scatter(x=dfl.index, y=dfl * 100)],
    'layout': go.Layout(title='Gross Leverage (%)', autosize=False, width=800)
})

In [ ]:
chart.spread('LBS2F12')